In [1]:
# 四半期の騰落率が大きく動いた銘柄の過去分析行う

In [16]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import KessanPl, PricelistFig, PricelistPl, MeigaralistPl, read_data
from lib_dataprocess import ShikihoPl, FinancequotePl, print_finance_quote, PricelistFig, KessanFig, IndexPricelistPl
import polars as pl
from datetime import date
from dateutil.relativedelta import relativedelta

RevPl = PricelistPl("reviced_pricelist.parquet")
RawPL = PricelistPl("raw_pricelist.parquet")
FPL = FinancequotePl()
FPL.with_columns_market_cap()
MPL = MeigaralistPl()
SPL = ShikihoPl()

In [3]:
#####
##### 評価日における四半期騰落率一覧
#####

In [4]:
valuation_date = date(2022, 4, 1)


KPl = KessanPl()
updown_rate_df = KPl.get_quater_settlement_price_updown_rate(valuation_date, "nh225")
updown_rate_df

code,start_date,end_date,updown_rate,nh_updown_rate
i64,date,date,f64,f64
1301,2022-02-05,2022-05-13,1.25,-3.29
1332,2022-02-05,2022-05-12,-0.19,-5.78
1333,2022-02-08,2022-05-09,-7.3,-3.66
1352,2022-01-29,2022-05-13,38.1,-0.98
1375,2022-02-04,2022-05-12,-14.7,-4.97
…,…,…,…,…
9993,2022-01-08,2022-04-12,-2.8,-7.21
9994,2022-02-11,2022-05-12,-2.08,-5.7
9995,2022-01-29,2022-05-16,-3.95,-0.54


In [5]:
#####
##### valuation_date時点で発表済四半期決算の業績成長率一覧(growth_rate_df)の作成
#####

In [6]:
KPL1 = KessanPl()
KPL2 = KessanPl()
KPL1.with_columns_growthrate_lastyear()
KPL2.with_columns_diff_growthrate()
KPL1.df = KPL1.df.filter(pl.col("settlement_type")=="四")
KPL2.df = KPL2.df.filter(pl.col("settlement_type")=="四")
df = KPL1.df.join(KPL2.df, on=["code", "settlement_date"], how="left")
KPL3 = KessanPl(df)
df =KPL3.get_latest_quater_settlements(valuation_date)
df = df.select([
    "code", "settlement_date", "announcement_date", "sales", "operating_income", "sales_growthrate", "diff_operating_income_growthrate"
])
df = df.rename({"settlement_date": "評価日発表済最新四半期決算"})
growth_rate_df = df
growth_rate_df

code,評価日発表済最新四半期決算,announcement_date,sales,operating_income,sales_growthrate,diff_operating_income_growthrate
i64,date,date,i64,i64,f64,f64
1301,2021-12-31,2022-02-04,79597,2493,2.7,9.2
1332,2021-12-31,2022-02-04,185668,10786,-5.7,-23.5
1333,2021-12-31,2022-02-07,249002,9913,-8.5,-12.6
1352,2021-12-31,2022-01-28,9619,609,-60.7,-0.6
1375,2021-12-31,2022-02-03,15954,2900,-6.5,83.2
…,…,…,…,…,…,…
9993,2021-11-30,2022-01-07,26555,32,-0.1,1438.9
9994,2021-12-31,2022-02-10,41072,1507,0.8,171.8
9995,2021-12-31,2022-01-28,18236,569,16.3,19.6


In [7]:
#####
##### 評価dfの作成：　updown_rate_dfとgrowth_rate_dfをjoinして必要な列を残す
#####

In [9]:
df = growth_rate_df.join(updown_rate_df, on=["code"], how="left")
df = df.with_columns([
    (pl.lit(100) * pl.col("operating_income") / pl.col("sales")).round(2).alias("営業利益率")
])
df = df.select(df.columns[:5] + [df.columns[-1]] + df.columns[5:-1])
df = df.select(df.columns[:2]+df.columns[3:])
df = df.sort(by=["updown_rate"], descending=[True])
df = df.drop_nulls()
evaluated_df = df
evaluated_df

code,評価日発表済最新四半期決算,sales,operating_income,営業利益率,sales_growthrate,diff_operating_income_growthrate,start_date,end_date,updown_rate,nh_updown_rate
i64,date,i64,i64,f64,f64,f64,date,date,f64,f64
8139,2021-12-31,4555,101,2.22,-11.8,22.2,2022-02-11,2022-05-13,377.39,-3.22
7901,2022-01-31,163,-85,-52.15,5.2,112.5,2022-03-10,2022-06-08,110.87,12.45
7066,2021-12-31,654,-75,-11.47,9.9,-210.2,2022-02-11,2022-05-13,98.55,-3.22
3094,2021-11-30,17407,-214,-1.23,-4.7,28.0,2022-01-15,2022-04-14,89.41,-4.1
4387,2021-12-31,805,-211,-26.21,22.7,-163.1,2022-02-15,2022-05-13,78.61,-2.78
…,…,…,…,…,…,…,…,…,…,…
2980,2021-12-31,3947,654,16.57,442.9,15.8,2022-02-01,2022-05-11,-53.79,-3.51
4169,2021-12-31,832,-97,-11.66,80.5,-18.3,2022-02-11,2022-05-13,-57.32,-3.22
3782,2021-12-31,280,-78,-27.86,-15.2,166.0,2022-02-11,2022-08-12,-60.71,4.55


In [10]:
#####
##### filter
#####

In [11]:
# para
min_sales = 5000
min_operating_income = 0
min_営業利益率 = 10
max_営業利益率 = 100
min_sales_growth_rate = 10
max_sales_growth_rate = 100
min_diff_operating_income_growthrate = 0

r1 = 1.5

# 単純filter
df = evaluated_df
df = df.filter(pl.col("sales")>=min_sales)\
    .filter(pl.col("operating_income")>=min_operating_income)\
    .filter(pl.col("営業利益率")>=min_営業利益率)\
    .filter(pl.col("営業利益率")<=max_営業利益率)\
    .filter(pl.col("sales_growthrate")>=min_sales_growth_rate)\
    .filter(pl.col("sales_growthrate")<=max_sales_growth_rate)\
    .filter(pl.col("diff_operating_income_growthrate")>=min_diff_operating_income_growthrate)

# 列間filter
df = df.filter(
    pl.col("sales_growthrate")<=(pl.lit(r1) * pl.col("diff_operating_income_growthrate"))
)

target_df = df
target_df


code,評価日発表済最新四半期決算,sales,operating_income,営業利益率,sales_growthrate,diff_operating_income_growthrate,start_date,end_date,updown_rate,nh_updown_rate
i64,date,i64,i64,f64,f64,f64,date,date,f64,f64
4331,2021-12-31,11647,1887,16.2,42.6,73.6,2022-02-11,2022-05-13,32.79,-3.22
3064,2021-12-31,51281,6588,12.85,20.7,15.4,2022-02-04,2022-04-28,32.03,-0.92
9273,2021-12-31,5510,1297,23.54,22.9,37.9,2022-02-11,2022-05-12,29.21,-5.7
2222,2021-12-31,10963,2278,20.78,30.5,45.5,2022-02-04,2022-05-13,28.16,-2.47
4293,2021-12-31,6572,1940,29.52,24.6,56.7,2022-02-11,2022-05-12,26.84,-5.7
…,…,…,…,…,…,…,…,…,…,…
3302,2021-12-31,12024,2641,21.96,46.5,36.0,2022-02-15,2022-05-12,-23.37,-5.28
7816,2021-12-31,7320,1263,17.25,37.0,28.5,2022-02-15,2022-05-13,-23.58,-2.78
7734,2021-12-31,9631,2364,24.55,20.0,55.0,2022-02-11,2022-05-12,-25.15,-5.7


In [12]:
#####
##### 統計結果出力
#####

In [13]:
df = evaluated_df
print("##### 全体")
print(f'総銘柄数：{df.shape[0]}')
a = df.filter(pl.col("updown_rate")>0).shape[0]
b = df.filter(pl.col("updown_rate")<=0).shape[0]
uprate = round(100*a/(a+b), 1)
print(f'上昇銘柄数:{a}／下落銘柄数:{b}／上昇率:({uprate}%)')
print(f'全銘柄平均騰落率:{round(df["updown_rate"].mean(), 2)}%')
print(f'日経225平均騰落率:{round(df["nh_updown_rate"].mean(), 2)}%')
print()

df = target_df
print("##### 抽出銘柄")
print(f'総銘柄数：{df.shape[0]}')
a = df.filter(pl.col("updown_rate")>0).shape[0]
b = df.filter(pl.col("updown_rate")<=0).shape[0]
uprate = round(100*a/(a+b), 1)
print(f'上昇銘柄数:{a}／下落銘柄数:{b}／上昇率:({uprate}%)')
print(f'抽出銘柄平均騰落率:{round(df["updown_rate"].mean(), 2)}%')
print(f'日経225平均騰落率:{round(df["nh_updown_rate"].mean(), 2)}%')

##### 全体
総銘柄数：3550
上昇銘柄数:1347／下落銘柄数:2203／上昇率:(37.9%)
全銘柄平均騰落率:-1.38%
日経225平均騰落率:-2.98%

##### 抽出銘柄
総銘柄数：189
上昇銘柄数:70／下落銘柄数:119／上昇率:(37.0%)
抽出銘柄平均騰落率:-3.31%
日経225平均騰落率:-3.05%


In [49]:
#####
##### 個別銘柄の分析
#####

In [29]:
# ファンダメンタルズ
code = 6920
print_finance_quote(RawPL.df, FPL.df, code, valuation_date=valuation_date)
print()
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)



レーザーテック(6920)の銘柄情報

終値: 20635.0円(2022年04月01日)


OutOfBoundsError: index 0 is out of bounds for sequence of length 0

In [30]:
#####
##### 四半期決算推移
#####

In [31]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [32]:
#####
##### 株価推移
#####

In [33]:
# para
delta = relativedelta(days=100)

In [34]:
# 日足
KPL = KessanPl()
df = KPL.df.filter(pl.col("announcement_date")<valuation_date).filter(pl.col("code")==code)
ann_date = df["announcement_date"].to_list()[-1]
print(f'決算発表日: {ann_date}')
start_date = ann_date - delta
end_date = ann_date + delta

PFIG = PricelistFig(code, RevPl.df, MPL.df, start_date, end_date)
# 決算発表日にvlineを引く
PFIG.add_vline_announcement_date()
PFIG.fig.show()

決算発表日: 2022-01-31
